In [1]:
# Import Packages
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
# Load the Dataset
emotion_data = pd.read_csv('./fer2013.csv')
emotion_data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
# prepare data
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in emotion_data.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

X_train = np.array(X_train, dtype = 'uint8')
y_train = np.array(y_train, dtype = 'uint8')
X_test = np.array(X_test, dtype = 'uint8')
y_test = np.array(y_test, dtype = 'uint8')

y_train= to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [4]:
# reshape data
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [5]:
# image augmentation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')

testgen = ImageDataGenerator(rescale=1./255)
datagen.fit(X_train)

batch_size = 64

In [6]:
# fit generator to data
train_flow = datagen.flow(X_train, y_train, batch_size=batch_size)
test_flow = testgen.flow(X_test, y_test, batch_size=batch_size)

In [7]:
# build and compile model using cnn
# build the model
def build_model(input_shape=(48,48,1), activation_fn = 'relu', learning_rate_fn = 0.0001): # if accuracy bad, declare function for each block and tune the batch size
    # first input model
    visible = Input(shape=input_shape, name='input')
    num_classes = 7
    # 1st block
    conv1_1 = Conv2D(64, kernel_size=3, activation= activation_fn, padding='same', name = 'conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation= activation_fn, padding='same', name = 'conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)
    # 2nd block
    conv2_1 = Conv2D(128, kernel_size=3, activation= activation_fn, padding='same', name = 'conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation= activation_fn, padding='same', name = 'conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation= activation_fn, padding='same', name = 'conv2_3')(conv2_2)
    conv2_2 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)
    # 3rd block
    conv3_1 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    conv3_4 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_4')(conv3_3)
    conv3_4 = BatchNormalization()(conv3_4)
    pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
    drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)
    # 4th block
    conv4_1 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_4 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_4')(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
    drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)
    # 5th block
    conv5_1 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    conv5_4 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_4')(conv5_3)
    conv5_3 = BatchNormalization()(conv5_3)
    pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
    drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)
    # flatten and output
    flatten = Flatten(name = 'flatten')(drop5_1)
    ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)
    # create model
    model = Model(inputs =visible, outputs = ouput)
    # summary layers
    print(model.summary())
    opt = Adam(learning_rate=learning_rate_fn, decay=1e-6) # if accruacy low, unite model opt in one function and tune the learning rate
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# compile model
model = build_model()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
conv1_2 (Conv2D)             (None, 48, 48, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
pool1_1 (MaxPooling2D)       (None, 24, 24, 64)        0         
_________________________________________________________________
drop1_1 (Dropout)            (None, 24, 24, 64)        0     

In [8]:
# train model
num_epochs = 100
history = model.fit_generator(train_flow,
        steps_per_epoch=len(X_train) / batch_size,
        epochs=num_epochs,
        verbose=1,
        validation_data=test_flow,
        validation_steps=len(X_test) / batch_size)

Epoch 1/100


/homes/iws/runsenwu/how-are-you-really/HAYR/lib64/python3.6/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


448/448 [==============================] - 218s 482ms/step - loss: 2.0274 - accuracy: 0.2166 - val_loss: 1.8856 - val_accuracy: 0.1817
Epoch 2/100
448/448 [==============================] - 214s 477ms/step - loss: 1.7834 - accuracy: 0.2615 - val_loss: 1.8031 - val_accuracy: 0.2680
Epoch 3/100
448/448 [==============================] - 217s 484ms/step - loss: 1.7278 - accuracy: 0.2974 - val_loss: 1.7385 - val_accuracy: 0.3380
Epoch 4/100
448/448 [==============================] - 216s 482ms/step - loss: 1.6591 - accuracy: 0.3393 - val_loss: 1.6877 - val_accuracy: 0.3689
Epoch 5/100
448/448 [==============================] - 217s 485ms/step - loss: 1.5854 - accuracy: 0.3762 - val_loss: 1.4952 - val_accuracy: 0.4179
Epoch 6/100
448/448 [==============================] - 217s 484ms/step - loss: 1.5133 - accuracy: 0.4101 - val_loss: 1.6847 - val_accuracy: 0.3929
Epoch 7/100
448/448 [==============================] - 216s 481ms/step - loss: 1.4556 - accuracy: 0.4386 - val_loss: 1.3703 - val_

In [1]:
#Hyper parameter tuning

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


def build_model(input_shape=(48,48,1), activation_fn = 'relu', learning_rate_fn = 0.0001): # if accuracy bad, declare function for each block and tune the batch size
    # first input model
    visible = Input(shape=input_shape, name='input')
    num_classes = 7
    # 1st block
    conv1_1 = Conv2D(64, kernel_size=3, activation= activation_fn, padding='same', name = 'conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation= activation_fn, padding='same', name = 'conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)
    # 2nd block
    conv2_1 = Conv2D(128, kernel_size=3, activation= activation_fn, padding='same', name = 'conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation= activation_fn, padding='same', name = 'conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation= activation_fn, padding='same', name = 'conv2_3')(conv2_2)
    conv2_2 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)
    # 3rd block
    conv3_1 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    conv3_4 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv3_4')(conv3_3)
    conv3_4 = BatchNormalization()(conv3_4)
    pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
    drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)
    # 4th block
    conv4_1 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_4 = Conv2D(256, kernel_size=3, activation= activation_fn, padding='same', name = 'conv4_4')(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
    drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)
    # 5th block
    conv5_1 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    conv5_4 = Conv2D(512, kernel_size=3, activation= activation_fn, padding='same', name = 'conv5_4')(conv5_3)
    conv5_3 = BatchNormalization()(conv5_3)
    pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
    drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)
    # flatten and output
    flatten = Flatten(name = 'flatten')(drop5_1)
    ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)
    # create model
    model = Model(inputs =visible, outputs = ouput)
    # summary layers
    print(model.summary())
    opt = Adam(learning_rate=learning_rate_fn, decay=1e-6) # if accruacy low, unite model opt in one function and tune the learning rate
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


# define models and parameters
# model = RandomForestRegressor() 
# n_estimators = [10, 100, 1000]  
batch_size = [32, 64, 96, 128, 256, 512]
activation_fuction = ['relu', 'sigmoid', 'tanh']
learning_rate = [0.0001, 0.0005, 0.001, 0.005]

# define grid search
# grid = dict(n_estimators=n_estimators)

#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=101)

model = KerasClassifier(build_fn = build_model) 

params={# 'batch_size':[32, 64, 96, 128, 256, 512], 
        'activation_fn':['relu', 'sigmoid', 'tanh'],
        'learning_rate_fn':[0.0001, 0.0005, 0.001, 0.005],
        }

gs = GridSearchCV(estimator=model, param_grid=params, cv=10)
# now fit the dataset to the GridSearchCV object. 
gs = gs.fit(X_train, y_train)

best_params=gs.best_params_
accuracy=gs.best_score_

# grid_search_forest = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, 
#                            scoring='r2',error_score=0,verbose=2,cv=2)

# grid_search_forest.fit(X_train_std, Y_train)

# summarize results
# print(f"Best: {grid_search_forest.best_score_:.3f} using {grid_search_forest.best_params_}")
# means = grid_search_forest.cv_results_['mean_test_score']
# stds = grid_search_forest.cv_results_['std_test_score']
# params = grid_search_forest.cv_results_['params']

# for mean, stdev, param in zip(means, stds, params):
#     print(f"{mean:.3f} ({stdev:.3f}) with: {param}")


NameError: name 'X_train' is not defined

In [ ]:
# save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")